In [1]:
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
from operator import itemgetter
import datetime

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


This file loads the behavioral data from the output file, matches it to the list of betas we have, and then splits it into train and test groups.

As long as input data doesn't change it shouldn't change which subjects are allocated to each group each time.

If input data does change, then you can use the previous output of this file to manually hardcode which groups already-classified subjects should be in using `already_peaked_paths` files: see below.

In [2]:
#load behavioral data

import dev_wtp_io_utils
behavioral_data_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(behavioral_data_path)



subject 040, subject 039, subject 168, subject 063, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of length 2

BidDuration: array of length 2

FoodPic: array 

In [3]:
#load design data from the official list of task design.
#this is NOT per subject.
betas = pd.read_csv("../data/DEV QC and Exclusions - betas_machine_readable.csv")
wtp_betas = betas[betas.task=='WTP betas']
#get the betas just for the four runs
wtp_betas_runfiles = wtp_betas[[bt in ['run1','run2','run3','run4'] for bt in wtp_betas.type]].copy()
#number them
wtp_betas_runfiles['event_id'] = wtp_betas_runfiles.groupby('type').cumcount()+1

In [4]:
wtpw1_behavdesign = wtp_betas_runfiles.merge(run_event_df_list, how='left',
                         left_on=['type','event_id'],
                        right_on=['run','event_id'])

wtpw1_behavdesign = wtpw1_behavdesign[(wtpw1_behavdesign.wave==1)]

wtpw1_behavdesign.sample(5)


,beta,type,task,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
1107,beta_0002.nii,run1,WTP betas,2,3.000000,18.646307,6.520440,Pistachios.bmp,2,healthy_liked,healthy,liked,4,6,1.780283,25.166746,run1,1,DEV022
9761,beta_0016.nii,run1,WTP betas,16,1.421652,168.756718,6.516436,lunchables_turkey.bmp,16,unhealthy_liked,unhealthy,liked,4,8,NaN,175.273155,run1,1,DEV060
21577,beta_0046.nii,run3,WTP betas,4,0.095117,39.681025,6.528325,CoolRanchDoritos.bmp,4,unhealthy_disliked,unhealthy,disliked,2,NULL,2.680966,46.209350,run3,1,DEV032
23320,beta_0051.nii,run3,WTP betas,9,1.584060,96.124035,6.525667,Strawberries.bmp,9,healthy_liked,healthy,liked,4,7,0.800571,102.649703,run3,1,DEV153
28543,beta_0071.nii,run4,WTP betas,8,2.083661,84.866294,6.520734,laffytaffyred_one.bmp,8,unhealthy_disliked,unhealthy,disliked,1,5,1.584060,91.387028,run4,1,DEV024


In [5]:
#iterate through subject directories.

subject_dirs_all = glob.glob('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/betaseries/sub-DEV*')
subject_dirs_all.sort()

In [6]:
#make some labels based on this list
import re
sub_labels_all = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs_all]
sub_ids_all = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs_all]

## Deal with incomplete data

1. Events listed in betas that do not have a betas file

2. Subjects listed in subject_dirs that don't have corresponding beta data

In [7]:
wtpw1_behavdesign

,beta,type,task,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
3,beta_0001.nii,run1,WTP betas,1,3.000000,6.080660,6.564707,SourPunch.bmp,1,unhealthy_liked,unhealthy,liked,4,NULL,3.0,12.645367,run1,1,DEV040
6,beta_0001.nii,run1,WTP betas,1,3.000000,6.068091,6.548297,M&Ms.bmp,1,unhealthy_liked,unhealthy,liked,4,6,3.0,12.616389,run1,1,DEV039
9,beta_0001.nii,run1,WTP betas,1,3.000000,6.076286,6.548571,MacNCheese.bmp,1,unhealthy_liked,unhealthy,liked,3,8,3.0,12.624857,run1,1,DEV168
13,beta_0001.nii,run1,WTP betas,1,3.000000,6.085906,6.553895,FamousAmos.bmp,1,unhealthy_liked,unhealthy,liked,2,6,3.0,12.639801,run1,1,DEV063
16,beta_0001.nii,run1,WTP betas,1,3.000000,6.068752,6.549177,PeanutButterCup.bmp,1,unhealthy_liked,unhealthy,liked,4,NULL,3.0,12.617930,run1,1,DEV129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31399,beta_0079.nii,run4,WTP betas,16,1.421652,168.783176,6.532094,lunchables_turkey.bmp,16,unhealthy_liked,unhealthy,liked,1,NULL,NaN,175.315270,run4,1,DEV179
31400,beta_0079.nii,run4,WTP betas,16,1.421652,168.726133,6.532007,FamousAmos.bmp,16,unhealthy_liked,unhealthy,liked,3,6,NaN,175.258140,run4,1,DEV098
31403,beta_0079.nii,run4,WTP betas,16,1.421652,168.782857,6.533177,SourCreamChips.bmp,16,unhealthy_liked,unhealthy,liked,4,7,NaN,175.316034,run4,1,DEV108
31404,beta_0079.nii,run4,WTP betas,16,1.421652,168.644515,6.516474,Poptart1.bmp,16,unhealthy_liked,unhealthy,liked,2,5,NaN,175.160989,run4,1,DEV035


In [8]:
pd.set_option('display.max_rows', 50)

Exclude from consideration subjects that have too few events (i.e., less than 64).

In [9]:
events_per_subject = wtpw1_behavdesign.groupby('subject').duration.count()
subjects_with_too_few_events = events_per_subject[events_per_subject<64].index.tolist()

wtpw1_behavdesign_clean = wtpw1_behavdesign.loc[[(s in subjects_with_too_few_events)==False for s in wtpw1_behavdesign.subject]]


In [10]:
print(wtpw1_behavdesign_clean.shape, wtpw1_behavdesign.shape)

(11648, 19) (11824, 19)


The following subjects have behavioral data recorded in the MATLAB behavioral output folder via the function `get_comprehensive_run_data_from_mat_dir`, but they do not have a beta.nii folder:

In [11]:

subjects_with_behavioral_no_beta_folder = set(wtpw1_behavdesign_clean.subject).difference(set(sub_labels_all))
subjects_with_behavioral_no_beta_folder = list(subjects_with_behavioral_no_beta_folder)
subjects_with_behavioral_no_beta_folder.sort()
print( subjects_with_behavioral_no_beta_folder)

['DEV002', 'DEV007', 'DEV011', 'DEV020', 'DEV032', 'DEV047', 'DEV063', 'DEV064', 'DEV067', 'DEV075', 'DEV078', 'DEV088', 'DEV090', 'DEV093', 'DEV094', 'DEV095', 'DEV096', 'DEV097', 'DEV098', 'DEV099', 'DEV100', 'DEV101', 'DEV102', 'DEV103', 'DEV104', 'DEV105', 'DEV106', 'DEV107', 'DEV108', 'DEV109', 'DEV110', 'DEV111', 'DEV112', 'DEV113', 'DEV115', 'DEV116', 'DEV117', 'DEV118', 'DEV119', 'DEV120', 'DEV121', 'DEV122', 'DEV123', 'DEV124', 'DEV126', 'DEV127', 'DEV128', 'DEV129', 'DEV130', 'DEV132', 'DEV133', 'DEV134', 'DEV135', 'DEV137', 'DEV138', 'DEV139', 'DEV140', 'DEV141', 'DEV143', 'DEV144', 'DEV145', 'DEV147', 'DEV149', 'DEV150', 'DEV151', 'DEV152', 'DEV153', 'DEV154', 'DEV155', 'DEV156', 'DEV157', 'DEV158', 'DEV159', 'DEV161', 'DEV163', 'DEV164', 'DEV166', 'DEV167', 'DEV168', 'DEV169', 'DEV170', 'DEV171', 'DEV173', 'DEV174', 'DEV176', 'DEV177', 'DEV178', 'DEV179', 'DEV180', 'DEV181', 'DEV182', 'DEV183', 'DEV185', 'DEV186', 'DEV187', 'DEV189', 'DEV190', 'DEV191', 'DEV192', 'DEV193',

The following subjects have a `beta.nii` folder but they do not have behavioral data recorded in the MATLAB behavioral output folder via the function `get_comprehensive_run_data_from_mat_dir`.

See: https://docs.google.com/presentation/d/1K-nFrZYE6rR8t0myNyacB7frBzV3B1--nMqPhVkwL8E/edit#slide=id.gd9fcc4129a_0_0

In [12]:
behavioral_data_subjects = set(wtpw1_behavdesign_clean.subject)

In [13]:
subjects_with_beta_folder_no_behavioral = set(sub_labels_all).difference(behavioral_data_subjects)

In [14]:
subjects_with_beta_folder_no_behavioral

{'DEV072', 'DEV082'}

Finally we want to remove those subjects with no behavioral data from the list...

In [15]:
subject_dirs_usable = [d for d, sl in zip(subject_dirs_all,sub_labels_all) if sl not in subjects_with_beta_folder_no_behavioral]
sub_labels_usable = [sl for d, sl in zip(subject_dirs_all,sub_labels_all) if sl not in subjects_with_beta_folder_no_behavioral]

In [16]:
ts = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')

In [17]:
added_labels = pd.DataFrame({"sub_label":sub_labels_usable})
added_labels['added'] = datetime.datetime.now()
added_labels.to_csv("../data/useable_subject_labels.csv",index=False)

### Manage split and select subjects

In [18]:
# #make some labels based on this list
# import re
# sub_labels = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs]
# sub_ids = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs]

Get the subjects we've already peaked at. These were _also_ chosen randomly, just via a prior process. They'll be part of the train group.

In [19]:
already_peaked_paths = pd.concat(
    [pd.read_csv("../data/train_subjs_25_20210601T131602.csv"),
    pd.read_csv("../data/holdout_subjs_5_20210601T131602.csv")]
)


already_peaked_labels = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in already_peaked_paths['0']]


In [20]:
pre_determined_training_subjs = already_peaked_labels

Determine the sizes of the groups we want to use:

In [21]:
total_sample_size = len(subject_dirs_usable)# - (len(subject_dirs_usable) % 5)
holdout_size = 5
holdout_n = (int)(total_sample_size/holdout_size)

In [22]:
holdout_n

15

In [23]:
print(total_sample_size,holdout_n,total_sample_size-holdout_n)

75 15 60


The only way to get a REALLY stable set of useable subjects, robust to even corrections to the dataset, is to make this list manually, so that's what we'll do.

In [24]:

ts = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')

In [25]:
ts

'20210719T114802'

In [26]:
test_train_df = pd.DataFrame({"sub_label":sub_labels_usable})

Now we get the pre-determined subjects marked as 'train' first

In [27]:
test_train_df['SplitGroup'] = None

In [28]:
for sl in test_train_df.sub_label:
    if sl in pre_determined_training_subjs:
        test_train_df.loc[test_train_df.sub_label==sl,'SplitGroup']='Train'
    

In [29]:
test_train_df.SplitGroup.value_counts()

Train    30
Name: SplitGroup, dtype: int64

And _now_ we can use the order of the subjects in the randomization to select remaining subjects to allocate to the two groups. 

Set a random order for the subjects to be selected from...

In [30]:
unallocated_subjects = test_train_df[test_train_df.SplitGroup.isnull()].sub_label.tolist()

In [31]:
random.seed(a=1622149277.6487303)
subj_labels_randomized = random.sample(unallocated_subjects,k=len(unallocated_subjects)) # add an extra 5 as a hold-out

In [32]:
holdout_subjs = subj_labels_randomized[0:holdout_n]


In [33]:
for hs in holdout_subjs:
    test_train_df.loc[test_train_df.sub_label==hs,'SplitGroup']='Holdout'

In [34]:
#mark the remaining subjects as for training
test_train_df.loc[test_train_df.SplitGroup.isnull(),'SplitGroup']='Train'

In [35]:
test_train_df.to_csv("../data/train_test_markers_"+ts+".csv",index=False)

In [36]:
test_train_df.SplitGroup.value_counts()

Train      60
Holdout    15
Name: SplitGroup, dtype: int64

In [37]:
test_train_df.shape

(75, 2)

In [38]:
wtpw1_behavdesign_clean.to_csv("../data/wtpw1_behavdesign_clean.csv",index=False)